In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import black


def print_res(old_d: dict, result, just_ans=True):
    d = old_d.copy()
    d["tools"] = eval(d["tools"])
    if just_ans:
        del d["tools"]
    d["answers"] = eval(d["answers"])
    print("-" * 12, end="\n\n\n")
    print(black.format_str(str(d), mode=black.Mode()))
    print("-" * 12, end="\n\n\n")
    print(result)

In [34]:
import re
from parser import parse


def handle_response(response):
    pattern = re.compile(r"(?s)```shonqol\s*([\s\S]*?)\s*```")
    matches = pattern.findall(response)

    # for code_block in matches:
    return parse(matches[0])

In [35]:
def calc_hit_fn_arg(answers, extracted):
    hit_fn = 0
    hit_arg = 0
    tot_args = 0

    # Filter out non-dictionary entries and entries missing keys
    valid_answers = [
        a for a in answers if isinstance(a, dict) and "name" in a and "arguments" in a
    ]
    valid_extracted = [
        e for e in extracted if isinstance(e, dict) and "name" in e and "arguments" in e
    ]

    # Create a lookup for extracted functions by name
    extracted_lookup = {e["name"]: e for e in valid_extracted}

    for a in valid_answers:
        fn_name = a["name"]
        if fn_name in extracted_lookup:
            hit_fn += 1
            ans_args = list(a["arguments"].values())
            tot_args += len(ans_args)
            ext_args = extracted_lookup[fn_name]["arguments"]
            for aarg in ans_args:
                if aarg in ext_args:
                    hit_arg += 1

    print("hit_fn", hit_fn, "out of", len(valid_answers))
    print("hit_arg", hit_arg, "out of", tot_args)
    return hit_fn, hit_arg


# Example data:
extracted = [
    {"name": "get_id", "arguments": ["Pulp Fiction"]},
    {"name": "search_torrents", "arguments": ["Pulp Fiction torrents"]},
    "Pulp Fiction torrents",
]


answers = [
    {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
    {
        "name": "search_torrents",
        "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
    },
]

calc_hit_fn_arg(answers, extracted)

hit_fn 2 out of 2
hit_arg 1 out of 3


(2, 1)

In [36]:
import pandas as pd

# df = pd.read_json("hf://datasets/Salesforce/xlam-function-calling-60k/xlam_function_calling_60k.json")
df = pd.read_json("./xlam_function_calling_60k.json")

In [37]:
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from prompts import main_prompt

prompt = ChatPromptTemplate.from_messages(
    [("system", main_prompt), ("user", "{query}")]
)
llm = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.7) | StrOutputParser()

In [39]:
d = df.iloc[6].to_dict()
result = llm.invoke(d)
print_res(d, result)

------------


{
    "id": 6,
    "query": "Identify the ID of 'Pulp Fiction' and search for torrents related to it.",
    "answers": [
        {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
        {
            "name": "search_torrents",
            "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
        },
    ],
}

------------


```shonqol
movieId = get_id("Pulp Fiction");
search_torrents(concat("Pulp Fiction, ", movieId));
```


In [40]:
import json


extracted_functions = handle_response(result)
print("*" * 20)
print((extracted_functions))
print("*" * 20)
print(d["answers"])
print("*" * 20)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

line 2:15 mismatched input '(' expecting '='
line 2:22 mismatched input '(' expecting '='
line 2:48 mismatched input ')' expecting '='


AttributeError: 'NoneType' object has no attribute 'value'

In [41]:
d = df.iloc[9].to_dict()
result = llm.invoke(d)
print_res(d, result)

------------


{
    "id": 9,
    "query": "Is 'https://www.apple.com' available in the Wayback Machine on September 9, 2015?",
    "answers": [
        {
            "name": "availability",
            "arguments": {"url": "https://www.apple.com", "timestamp": "20150909"},
        }
    ],
}

------------


```shonqol
availability("https://www.apple.com", "20150909");
```


In [43]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "availability",
        "arguments": [
            "https://www.apple.com",
            "20150909"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 2 out of 2


line 1:49 extraneous input ';' expecting {<EOF>, Identifier}


(1, 2)

In [44]:
d = df.iloc[103].to_dict()
result = llm.invoke(d)
print_res(d, result, just_ans=False)

------------


{
    "id": 103,
    "query": "What is the recommended daily calorie intake and macronutrient distribution for a 30-year-old male, weighing 75 kg, 180 cm tall, moderately active, aiming to maintain weight?",
    "answers": [
        {
            "name": "calculate_calorie_intake",
            "arguments": {
                "weight_kg": 75,
                "height_cm": 180,
                "age": 30,
                "sex": "male",
                "activity_level": 3,
                "goal": "maintain",
            },
        }
    ],
    "tools": [
        {
            "name": "calculate_calorie_intake",
            "description": "Calculates the recommended daily calorie intake and macronutrient distribution based on personal characteristics and goals.",
            "parameters": {
                "weight_kg": {
                    "description": "Body weight in kilograms.",
                    "type": "float",
                },
                "height_cm": {"descript

In [45]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "calculate_calorie_intake",
        "arguments": [
            75,
            180,
            30,
            "male",
            3,
            "maintain"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 6 out of 6


line 1:60 extraneous input ';' expecting {<EOF>, Identifier}


(1, 6)

In [46]:
d = df.iloc[586].to_dict()
result = llm.invoke(d)
print_res(d, result, just_ans=True)

------------


{
    "id": 586,
    "query": "I'm curious about the neuronal activity rate when the synaptic input rate is 100 per second, with a synaptic weight of 0.5 and a decay constant of 0.05. Also, could you check if 17 is a prime number and solve the quadratic equation 2x^2 - 3x + 1 = 0?",
    "answers": [
        {
            "name": "neuronal_activity_rate",
            "arguments": {
                "synaptic_input_rate": 100,
                "synaptic_weight": 0.5,
                "decay_constant": 0.05,
            },
        },
        {"name": "is_prime", "arguments": {"num": 17}},
        {"name": "solve_quadratic", "arguments": {"a": 2, "b": -3, "c": 1}},
    ],
}

------------


```shonqol
neuronal_activity = neuronal_activity_rate(100, 0.5, 0.05);
is_seventeen_prime = is_prime(17);
quadratic_roots = solve_quadratic(2, -3, 1);
```


In [50]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "neuronal_activity_rate",
        "arguments": [
            100,
            0.5,
            0.05
        ]
    },
    {
        "name": "is_prime",
        "arguments": [
            17
        ]
    },
    {
        "name": "solve_quadratic",
        "arguments": [
            2,
            -3,
            1
        ]
    }
]
hit_fn 3 out of 3
hit_arg 7 out of 7


(3, 7)

In [ ]:
d = df.iloc[998].to_dict()
result = llm.invoke(d)
print_res(
    d,
    result,
)

In [ ]:
import json

for _, row in df.iterrows():

    v = json.loads(str(row["answers"]))
    if not isinstance(v, list):
        continue
    if len(v) > 3:
        d = row.to_dict()
        break
result = llm.invoke(d)
print_res(d, result, just_ans=True)

In [ ]:
d = df.iloc[6].to_dict()
result = llm.invoke(d)
print_res(d, result)